Notebook for hacking and development on the Fispy project.

In [ ]:
import numpy as np
import datetime as dt
import pandas as pd
from dateutil.relativedelta import relativedelta

from bokeh.charts import Bar, output_file, show
from bokeh.charts.attributes import cat, color
from bokeh.charts.operations import blend
from bokeh.charts.utils import df_from_json
from bokeh.plotting import output_notebook, Figure
from bokeh.models import ColumnDataSource, HoverTool, TapTool, Quad

#from fispy.fispy import Asset, Portfolio

output_notebook()

### Testing for adding real stock data functions to investment assets 

In [ ]:
from pandas_datareader import data, wb

    
class Investment(object):
    def __init__(self, d):
        self.value = None
        self.roi = None
        self.dividend = None
        self.month_index = 0
        self.period = d['period']
        self.symbols = d['symbols']
        self.historical = []
    
    def buyStock(self, addQuantity):
        self.value += addQuantity
    
    def dividend_reinvest(self):
        self.value += self.dividend
    
    def appreciate(self):
        self.dividend = self.value * self.roi

    def add_historical_growth(self):
        delta = self.historical.values[self.month_index + 1]/self.historical.values[self.month_index]
        self.value = self.value * delta
        self.month_index += 1
    
    def fetch_real_stock_data(self):
        ls_key = 'Adj Close' 
        end = dt.datetime(year=dt.datetime.now().year,
                            month=dt.datetime.now().month, day=1)
        start = end - relativedelta(months=self.period + 2) 
        f = data.DataReader(self.symbols, 'yahoo', start, end)
        cleanData = f.ix[ls_key]
        df = pd.DataFrame(cleanData)
        self.historical = df.asfreq('m', method='pad',how='end')  

In [ ]:
t = Investment(d=d)

In [ ]:
t.fetch_real_stock_data()

In [ ]:
t.add_historical_growth()

In [ ]:
t.value

In [ ]:
fac = t.historical.values[1]/t.historical.values[0]

In [ ]:
fac * 10

In [ ]:
fac

In [ ]:
t.historical.values[1], t.historical.values[0]

In [ ]:
a1 = Asset(**{'kind':'job','monthly_income':1.5, 'monthly_expenses':0.7})


In [ ]:
a1 = Asset(**{'kind':'job','monthly_income':1.5, 'monthly_expenses':0.7})    


## Asset and Portfolio classes for hacking/testing ##

In [ ]:
class Asset(object):
    def __init__(self, **kwargs):
        self.kind = kwargs.get('kind')
        assert self.kind.lower() in [None, 'real estate', 'stocks', 'job', 'cash']
        self.monthly_income = kwargs.get('monthly_income')
        self.monthly_expenses = kwargs.get('monthly_expenses')
        self.start_date = kwargs.get('start_date', dt.datetime.now().date())
        self.end_date = kwargs.get('end_date')
        self.debt = kwargs.get('debt')
        self.value = kwargs.get('value')
        self.max_cash = kwargs.get('max_cash') 
        self.monthly_repayment = kwargs.get('monthly_repayment')
        self.pay_debt_asap = kwargs.get('pay_debt_asap')
    
    def __str__(self):
        return str("Asset = {0}".format(self.kind))

    def __iter__(self):
        for key, value in sorted(a1.__dict__.items()):
            yield key, value
    

class Portfolio(object):
    """ Operates on lists of assets. Can add assets individually
    with addNewAsset() function, or can initilize object with a
    list of asset objects.
    
    Add a test that at most only one cash asset (a summary asset) is given.
    """
    
    def __init__(self, *assets):
        self.assets = []
        print("{0} items passed".format(len(assets)))
        if assets:
            for asset in assets:
                self.assets.append(asset)
            self.date = min([asset.start_date for asset in self.assets])
        self.monthly_income = 0
        self.monthly_expenses = 0
        self.netInvestments = 0
        self.debt = 0
        self.prd = 60
        self.cash = 0
        self.networth = 0 # (cash + investments + property value) - debt
        
    
    def summary(self):
        print("--Summary--\nIncome: {0:3.2f} \nCash: {1:3.2f} \nDebt: {3:3.2f} \nInvestments: {2:3.2f}".format(
                self.monthly_income, self.cash, self.netInvestments, self.debt))
        print("Net worth: {0:3.2f}".format(self.networth))
    
    def addNewAsset(self, new_asset):
        self.assets.append(new_asset)
    
    def monthlyIncome(self):
        tmp_income = 0
        for asset in self.assets:
            if asset.monthly_income:
                tmp_income += asset.monthly_income
        self.monthly_income = tmp_income
    
    def monthlyExpenses(self):
        tmp_expenses = 0
        for asset in self.assets:
            if asset.monthly_expenses:
                tmp_expenses += asset.monthly_expenses
        self.monthly_expenses = tmp_expenses
        assert tmp_expenses < self.monthly_income,"Error: spending too much money..."
        self.monthly_income -= tmp_expenses
    
    def investmentPortfolio(self):
        tmp_net = 0
        for asset in self.assets:
            if asset.kind.lower() == 'stocks':
                # Work out value of assets
                tmp_net += asset.value
                # Increment assets (this should be replaced with historical monthly flux)
                asset.value *= 1.00333  # placeholder way of incrementing value of an asset
                # Buy more assets (at the moment this is naieve, 
                # and will spend all money on first stock it encounters in asset list)
                if self.monthly_income > 0: # If money left at end of month, invest it
                    asset.value += self.monthly_income
                    self.monthly_income = 0
        self.netInvestments = tmp_net
    
    def countCash(self):
        """nb. this function only expects to find one cash asset"""
        max_cash = 0
        for asset in self.assets:
            if asset.kind.lower() == 'cash': 
                self.cash = asset.value
                max_cash = asset.max_cash
                if self.cash < max_cash and self.monthly_income > 0:
                    asset.value += self.monthly_income
                    self.monthly_income = 0.0
                    self.cash = asset.value
                    
    def monthlyRepay(self):
        for asset in self.assets:
            if asset.debt and asset.monthly_repayment:
                assert self.monthly_income > asset.monthly_repayment,"Error: cant meet monthly repayment :("
                if (asset.debt - asset.monthly_repayment) <= 0.0:
                    #if this is the last payment...
                    self.monthly_income -= asset.debt
                    asset.debt = None
                    asset.monthly_repayment = None
                    return
                else:
                    if asset.pay_debt_asap:
                        asset.debt -= self.monthly_income
                        self.monthly_income = 0.0
                    else:
                        asset.debt -= asset.monthly_repayment
                        self.monthly_income -= asset.monthly_repayment
    
    def monthlyDebt(self):
        tmp_debt = 0
        for asset in self.assets:
            if asset.debt:
                tmp_debt += asset.debt
        self.debt = tmp_debt
        
    
    def calcNetWorth(self):
        net_value = 0
        for asset in self.assets:
            # cash, property and investments all have a value property (not jobs)
            if asset.value:
                net_value += asset.value
        self.networth = net_value - self.debt
    
    def update_monthly(self):
        self.date = self.date + relativedelta(months=1)
        self.monthlyIncome()   # Gather income at start of month
        self.monthlyExpenses() # Work out living expenses and subtract it from the income
        self.monthlyRepay()    # Repay monthly morgage expenses from income
        self.monthlyDebt()     # Work out remaining size of accumulated debt
        self.countCash()       # Count the cash and add to pile if required
        self.investmentPortfolio() # Gather investments value, and grow, also buy more if money left
        self.calcNetWorth()
        
    def quad_positions(self, left, right, bottom, top, color):
        # Add debt marker
        left.append(self.date)
        right.append(self.date + relativedelta(months=1))
        top.append(0.0)
        bottom.append(self.debt * -1)
        color.append('#FE642E')
        # Add cash marker
        left.append(self.date)
        right.append(self.date + relativedelta(months=1))
        bottom.append(0.0)
        top.append(self.cash)
        color.append('green')
        # Add stock marker
        left.append(self.date)
        right.append(self.date + relativedelta(months=1))
        bottom.append(self.cash)
        top.append(self.cash + self.netInvestments)
        color.append('#BF00FF')
        # Add net worth marker (including  primary property)
        left.append(self.date)
        right.append(self.date + relativedelta(months=1))
        bottom.append(self.cash + self.netInvestments)
        top.append(self.networth)
        color.append('#BDBDBD')
        return

    def gen_quads(self):
        left = []
        right = []
        top=[]
        bottom=[]
        color=[]
        for i in range(self.prd):
            self.update_monthly()
            self.quad_positions(left=left, top=top, bottom=bottom,
                                right=right, color=color)
        return pd.DataFrame({'left': left, 'right': right, 'top': top,
                            'bottom': bottom, 'color': color})

In [ ]:
# Initilise with list of assets...
d1 = {'kind': 'job', 
      'monthly_income': 1.5, 
      'start_date': dt.date(2016, 6, 1)}

d2 = {'kind': 'job',
      'monthly_income': 1.5,
      'monthly_expenses': 0.7,
      'start_date': dt.date(2016, 12, 1)}

d3 = {'kind': 'real estate',
      'debt': 70,
      'value':150,
      'monthly_repayment':0.5,
      'start_date': dt.date(2016, 6, 1),
      'pay_debt_asap':True}

d4 = {'kind': 'stocks',
      'value': 15,
      'symbol': 'NYSE:BRK.B'}

d5 = {'kind': 'cash',
      'value': 15,
      'max_cash':30}

test2 = Portfolio(Asset(**d1), Asset(**d2), Asset(**d3), Asset(**d4), Asset(**d5))

#test2.monthlyIncome()
#test2.monthlyExpenses()
#test2.monthlyDebt()
#print(test2.monthly_income, test2.monthly_expenses, test2.debt)
#print(test2.date)
bdf_quad = test2.gen_quads()
test2.summary()

In [ ]:
a1 = Asset(**{'kind':'job','monthly_income':1.5, 'monthly_expenses':0.7})
p1 = Portfolio(a1)

for k,v in a1:
    print(k,v)

In [ ]:
a1.debt

In [ ]:
source=ColumnDataSource(bdf_quad)
TOOLS="crosshair, pan, reset, resize, wheel_zoom"

plot = Figure(tools=TOOLS, x_axis_type='datetime', plot_height=500, 
              plot_width=700, title="FI chart")

#plot.add_tools(TapTool())
plot.yaxis.axis_label = "Val"
plot.xaxis.axis_label = "Time"

renderer = plot.quad(source=source, left='left', right='right', 
                     bottom='bottom', top='top', color='color')

#selected_q = Quad(fill_alpha=1, fill_color="green", line_color=None)
#nonselected_q = Quad(fill_alpha=0.2, fill_color="blue", line_color="firebrick")
#renderer.selection_glyph = selected_q
#renderer.nonselection_glyph = nonselected_q
    
show(plot)

In [ ]:
d1 = {'kind': 'job', 
      'monthly_income': 1.5, 
      'start_date': dt.date(2016, 6, 1)}

d2 = {'kind': 'job', 
      'monthly_income': 1.5, 
      'start_date': dt.date(2016, 6, 1)}


test2 = Portfolio(Asset(**d1),Asset(**d2) )
test2.monthlyIncome()

assert test2.monthly_income == 3.0, "Income incorrect"

In [ ]:
import datetime as dt

In [ ]:
tdate = dt.date(2003,2,1)
start = dt.date(2000,1,1)
end = dt.date(2099,1,1)
in_month = tdate >= start and tdate <= end